In [ ]:
import time
from urllib.request import urlopen #python 내장 라이브러리
import urllib
import bs4 
import pandas as pd
import numpy as np
from datetime import datetime
from urllib.error import HTTPError
import http.client
import torch
import pymysql
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings("ignore")
import sqlalchemy
from konlpy.tag import Mecab
#from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
from konlpy.tag import Okt
from PIL import Image
import konlpy
import re




######뉴스요약######

##1. Crawling

print('start!')


print('크롤링 수행시간 :', time.strftime('%H시 %M분 %S초',time.localtime(time.time())))
today = str(datetime.today().strftime("%Y%m%d"))

global url_list
url_list = []
p = 'https://media.naver.com/press/'
oid_table = ['032','005','020','021','081','022','023','025','028','469']
sid_table = ['100', '101', '102', '104', '105']
for oid_t  in oid_table:
    oid = oid_t+'?'
    for i in sid_table:
        sid = 'sid='+ f"{i}"
        url = p+oid+sid
        html = urllib.request.urlopen(url)
        bs = bs4.BeautifulSoup(html, 'html.parser')
        #print(bs)
        # 언론사별 총 링크
        list = bs.findAll('a',{'class':'press_edit_news_link _es_pc_link'})
        for n, txt in enumerate(list):
            url_list.append(txt['href'])
print(url_list)





##2. Data Parshing / Save as DataFrame 

news_df = pd.DataFrame(columns= ['headline','writer','url','content','created_date','press_id','section_id'])

for url_l in url_list:
    url = url_l
    html = urllib.request.urlopen(url)
    bs = bs4.BeautifulSoup(html, 'html.parser')

    #파싱작업 
    try:
        #언론사
        press = bs.find('a',{'class':'media_end_head_origin_link'})
        pressid = eval(press['data-extra']).get('lk').get('oid')
        #print('언론사', eval(press['data-extra']).get('lk').get('oid'))

        #섹션(다중분류시 첫번째 분류 카테고리 채택)
        split_sections = bs.find('meta',{'name':"napp-site-analysis"})
        idx= split_sections['content'].find('section=')+8
        #print('섹션', split_sections['content'][idx:idx+3])


        #기자명
        reporter = bs.find('em',{'class','media_journalistcard_summary_name_text'})
        #print('기자 :', reporter.text)


        #입력일시
        date_time = bs.find('span',{'class':'media_end_head_info_datestamp_time _ARTICLE_DATE_TIME'})
        #print('입력일시 : ',date_time['data-date-time'])


        #헤드라인
        headline1 = bs.find('h2',{'class','media_end_head_headline'})
        headline = headline1.find('span').text
        #print('헤드라인 :',headline)


        #본문 (부제목, 기자명 제거 필요)
        content = bs.find('div',{'class','go_trans _article_content'})
        content = content.text.replace('\n', '').replace('\t', '')
        #print('본문 :', content)

        
        #dataframe에 저장
        # ['id','headline','writer','url','content','created_date','press_id','section_id'])
        news_data = {'press_id': str(eval(press['data-extra']).get('lk').get('oid')),
                    'section_id': str(split_sections['content'][idx:idx+3]),
                    'writer': reporter.text,
                    'created_date' : date_time['data-date-time'],
                    'headline': headline,
                    'content' : content,
                    'url': url}
        
        news_df = news_df.append(news_data, ignore_index=True)
        
    #404 error
    except HTTPError:
        continue
    #IncompleteRead error
    except http.client.IncompleteRead:
        #html = e2.partial
        continue
    #none-type error
    except Exception:
        continue

#dataframe 저장
today = str(datetime.today().strftime("%Y%m%d"))
news_df = news_df[(news_df['section_id'].isin(['100','101', '102', '104', '105']))] #section_id 필터링
news_df = news_df.drop_duplicates(subset = ['content'])
# idx = news_df[(news_df['writer'].str.len() >= 15)].index
# for i in idx:
#     news_df['writer'].loc[i] = news_df['writer'].loc[i][:news_df['writer'].loc[i].find(',')]
news_df.to_csv('/home/lab01/News_'+today+'.csv', index= True, index_label= ['id'])





##3. 토크나이저 및 모델 로드
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')




##4. GPU 사용가능 여부 확인 및 Device 설정

if torch.cuda.is_available():
    # GPU를 디바이스로 설정
    device = torch.device("cuda")
else:
    # CPU를 디바이스로 설정
    device = torch.device("cpu")
    
    
    
    

## 5. 모델적용
today = str(datetime.today().strftime("%Y%m%d"))
crawling_news_data = pd.read_csv('/home/lab01/News_'+today+'.csv',dtype= {'press_id': 'object', 'section_id': 'object'})
max_length = 512  # 최대 토큰 수

for i in range(0,len(crawling_news_data)):
    text = crawling_news_data['content'].loc[i]
    text = text.replace('\n', ' ')
    model.to(device)
    raw_input_ids = tokenizer.encode(str(text))
    input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

    # 입력 문장 길이 제한
    if len(input_ids) > max_length:
        input_ids = input_ids[:max_length]
    
    summary_ids = model.generate(torch.tensor([input_ids]).to(device), repetition_penalty=2.0, num_beams=4, 
                                 min_length = 50, max_length=250,  eos_token_id=1, early_stopping= True)
    summary_news = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
    crawling_news_data['content'].loc[i] = summary_news

crawling_news_data.to_csv('/home/lab01/News_s'+today+'.csv', index= False, index_label= ['id'])
print('end1!')
    


##6. DB 저장
today = str(datetime.today().strftime("%Y%m%d"))

df = pd.read_csv('/home/lab01/News_s'+today+'.csv',dtype= {'press_id': 'object', 'section_id': 'object'})
    
#DB 정보 입력 필요  
host = '**.***.***.**'
port= '****'
database = '***'
user = '****'
password = '****'

# DB 접속 엔진 객체 생성
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}', encoding='utf-8')

# DB 테이블 명
table_name = "news_article"


with engine.connect() as conn:
    # SQL DELETE 문 실행하여 데이터 삭제
    delete_query = sqlalchemy.text("DELETE FROM news_article")
    conn.execute(delete_query)


# DB에 DataFrame 적재
df.to_sql(index = False,
          name = table_name,
          con = engine,         
          if_exists = 'append'          
         )



######키워드추출######

df = pd.read_csv('/home/lab01/News_s'+today+'.csv',dtype= {'press_id': 'object', 'section_id': 'object'})



#정규표현식으로 한글만 필터링 (숫자제거)
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    
    result = hangul.sub('', text)
    return result

sec = ['000','100','101','102','104','105']

final = pd.DataFrame()
for i in sec:
    temp = pd.DataFrame(columns=['word','count','date','section_id'])
    df = pd.read_csv('/home/lab01/News_s'+today+'.csv',dtype= {'press_id': 'object', 'section_id': 'object'})
    if i == '000':
        df = pd.read_csv('/home/lab01/News_s'+today+'.csv',dtype= {'press_id': 'object', 'section_id': 'object'})
    else:
        df = df[df['section_id']== i]
    df['content'] = df['content'].apply(lambda x : text_cleaning(x)) 
    mecab = Mecab() #형태소 분석기 Mecab
    nouns = df['content'].apply(mecab.nouns)
    nouns = nouns.explode()

    df_word = pd.DataFrame({'word' : nouns})
    df_word['count'] = df_word['word'].str.len()
    df_word = df_word.query('count >= 2') #2글자이상의 단어만 추출
    df_word.sort_values(by= ['count'], ascending = False)
    df_word = df_word.groupby('word', as_index = False).count().sort_values(by= ['count'], ascending = False)
    
    #불용어제거
    stop_words = pd.read_csv('/home/lab01/stopwords.csv')
    stop_words = stop_words['불용어']
    result =[]
    for j in range(len(df_word['word'])):
        txt = df_word['word'][j]
        stop_words = set(stop_words)
        word_tokens = mecab.morphs(txt)
        result += [word for word in word_tokens if not word in stop_words]


    df_word = df_word[(df_word['word'].isin(result))]

    temp['word'] = df_word['word']
    temp['count'] = df_word['count']
    temp['date'] = today
    temp['section_id'] = str(i)
    
    final = pd.concat([final, temp], axis= 0)

keyword_db = final.sort_values(by=['count'], ascending = False).groupby('section_id').head(10).sort_values(by=['section_id'])
keyword_db.index = range(1, len(keyword_db)+1)
keyword_db.rename_axis('id').reset_index().to_csv('/home/lab01/keyword_'+today+'.csv', index= False)
    

#DB저장
df = pd.read_csv('/home/lab01/keyword_'+today+'.csv',dtype = {'section_id':'object'})

# DB 접속 엔진 객체 생성
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}', encoding='utf-8')

# DB 테이블 명
table_name = "news_keyword" 

with engine.connect() as conn:
    # SQL DELETE 문 실행하여 데이터 삭제
    delete_query = sqlalchemy.text("DELETE FROM news_keyword") 
    conn.execute(delete_query)


# DB에 DataFrame 적재
df.to_sql(index = False,
          name = table_name,
          con = engine,         
          if_exists = 'append'          
         )


print('end!')


start!
크롤링 수행시간 : 11시 07분 45초
['https://n.news.naver.com/article/032/0003228843?sid=100', 'https://n.news.naver.com/article/032/0003228842?sid=100', 'https://n.news.naver.com/article/032/0003228834?sid=100', 'https://n.news.naver.com/article/032/0003228830?sid=100', 'https://n.news.naver.com/article/032/0003228822?sid=100', 'https://n.news.naver.com/article/032/0003228817?sid=100', 'https://n.news.naver.com/article/032/0003228815?sid=100', 'https://n.news.naver.com/article/032/0003228746?sid=100', 'https://n.news.naver.com/article/032/0003228745?sid=100', 'https://n.news.naver.com/article/032/0003228744?sid=100', 'https://n.news.naver.com/article/032/0003228743?sid=100', 'https://n.news.naver.com/article/032/0003228742?sid=100', 'https://n.news.naver.com/article/032/0003228741?sid=100', 'https://n.news.naver.com/article/032/0003228740?sid=100', 'https://n.news.naver.com/article/032/0003228715?sid=100', 'https://n.news.naver.com/article/032/0003228714?sid=100', 'https://n.news.naver.com